In [9]:
import fitz
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from docx import Document
from langchain_ollama import OllamaLLM
import uuid
import hashlib

In [2]:
def get_pdf(file):
    text = ""
    if file.endswith(".pdf"):
        document = fitz.open(file)

        for page in document:
            text+=page.get_text()
    elif file.endswith(".DOCX"):
        document = Document(file)
        for page in document.paragraphs:
            text+=page.text+'\n'
    return text

In [13]:
print(get_pdf('PGP-2425-PR-2 (REL).DOCX'))


Planeamento e Gestão de Projeto
Ano Letivo 2024/2025
Tema do Projeto
Relatório da Etapa 2
Grupo YY
Nome 1	99999
Nome 2	99999
Nome 3	99999
Nome 4	99999
Nome 5	99999
Nome 6	99999

Índice
Introdução	3
1. Recursos	3
1.1. Pessoas da equipa	3
1.2. Ferramentas de desenvolvimento	3
1.3. Software para o sistema	3
2. Estimativas	4
2.1. Esforço disponível	4
2.2. Dados históricos	4
2.3. Estimativas baseadas em linhas de código	4
2.4. Estimativas baseadas no modelo COCOMO	4
3. Processo de desenvolvimento de software	5
4. Organização da equipa	5
5. Planeamento do projeto	5
5.1. Work packages	5
5.2. Calendarização	6
6. Gestão de riscos	6
6.1. Lista de riscos	6
6.2. Tabela de riscos	6
6.3. Plano RMMM	6
Conclusão	6
Bibliografia	7
Apêndice	8


Introdução
Descrição do contexto, problema a resolver e sua importância, objetivos e mais valias do projeto, e síntese dos aspetos cobertos no relatório. Deve ser citado o enunciado do projeto, pois é aí que estão os requisitos de informação, funcionais, e não fu

In [3]:
def get_pdf(file):
    document = fitz.open(file)
    text=""
    for page in document:
        text+=page.get_text()
    return text

In [3]:
print(get_pdf('test.pdf'))

test
aici testez filurile
test
1



In [3]:
def chunking(text):
    text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name="cl100k_base",chunk_size=500,chunk_overlap=50)
    texts=text_splitter.split_text(text)
    return texts

In [7]:
text=get_pdf('T04_Camada_Rede.pdf')
chunk=chunking(text)
chunk

['T04- Camada rede\nSubjects\nRC\nServiços da camada de rede\nTransporte de datagramas entre computadores\nOs protocolos desta camada correm nos nós terminais e nos routers \nOs routers examinam os cabeçalhos IP de todos os datagramas\nDuas funções chave\nRouting → Determine o caminho origem-destino , Ação global\nForwarding → Mover pacotes que chega no router para a saída apropriada; \nAção local no router\nAnalogia: \nO routing e planeamento de uma viagem\nO forwarding e a passar um cruzamento \nRouting vs forwarding \nalgoritmo de routing → Determina a rota através da rede\ntabela forwarding local → Determina como o pacote deve ser encaminhado neste \nrouter \nPlano de dados e Plano de controlo\nPlano de dados:\nPlano de controlo:\nT04- Camada rede\n1\nfunção local em cada router\ndetermina o caminho de \ndatagrama e a saida\nencaminhamento\nlogica global\nrouting e definido aqui\nduas abordagens:\ntradicional : algoritmos de \nrouting implementados nos \nrouters\nredes definidas po

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')
def transformer(chunk):

    embeddings = model.encode(chunk).tolist()
    return embeddings

In [8]:
transf=transformer(chunk)
transf

[[0.06689008325338364,
  -0.05782294273376465,
  -0.10100003331899643,
  -0.06162252649664879,
  -0.021763045340776443,
  -0.022860918194055557,
  -0.05662044510245323,
  0.0977248102426529,
  -0.005368255544453859,
  0.06166989728808403,
  0.06124408170580864,
  0.025569405406713486,
  -0.051362235099077225,
  -0.04431716352701187,
  0.03133014589548111,
  0.03478822484612465,
  0.05036143213510513,
  -0.06460617482662201,
  -0.024991342797875404,
  -0.03814813494682312,
  0.04923682659864426,
  -0.03889564052224159,
  -0.04140930250287056,
  -0.02542641945183277,
  -0.08234009891748428,
  -0.004899832420051098,
  0.0447823703289032,
  -0.006557965185493231,
  -0.057306960225105286,
  -0.03889065980911255,
  -0.007230847608298063,
  0.02356426604092121,
  -0.016044441610574722,
  0.050584252923727036,
  -0.10020457953214645,
  -0.05092183127999306,
  0.04244140163064003,
  -0.04034380614757538,
  -0.08184924721717834,
  0.07856937497854233,
  -0.07015880197286606,
  0.0144573794677853

In [5]:
def vector_db(chunks,chunk_transformed):
    client=chromadb.Client()
    try:
        collection=client.create_collection('my_collection')
        print("created collection")
    except Exception as e:
        collection=client.get_collection('my_collection')

    for i,embedding in enumerate(chunk_transformed):
        collection.add(
            documents=[chunks[i]],
            embeddings=[embedding],
            ids=[str(i)]
        )

    return collection

In [6]:
text=get_pdf('Laboratory 6.1-6.2.pdf')
chunks=chunking(text)
chunks_transformed=transformer(chunks)
vector=vector_db(chunks,chunks_transformed)

print(vector)

created collection
Collection(name=my_collection)


In [19]:
text = get_pdf("T04_Camada_Rede.pdf")
print(f"Text extras (primii 300 caractere): {text[:300]}...")
print("---------------------------")
chunks = chunking(text)
print(f"Număr chunk-uri: {len(chunks)}")
print("---------------------------")
print(f"Primul chunk: {chunks[0]}")

embeddings = transformer(chunks)
print("---------------------------")
print(f"Dimensiune embedding primul chunk: {len(embeddings[0])}")
print("---------------------------")

collection = vector_db(chunks, embeddings)
print(f"Număr documente în colecție: {len(collection.get(include=["documents"])['documents'])}")

Text extras (primii 300 caractere): T04- Camada rede
Subjects
RC
Serviços da camada de rede
Transporte de datagramas entre computadores
Os protocolos desta camada correm nos nós terminais e nos routers 
Os routers examinam os cabeçalhos IP de todos os datagramas
Duas funções chave
Routing → Determine o caminho origem-destino , Ação gl...
---------------------------
Număr chunk-uri: 9
---------------------------
Primul chunk: T04- Camada rede
Subjects
RC
Serviços da camada de rede
Transporte de datagramas entre computadores
Os protocolos desta camada correm nos nós terminais e nos routers 
Os routers examinam os cabeçalhos IP de todos os datagramas
Duas funções chave
Routing → Determine o caminho origem-destino , Ação global
Forwarding → Mover pacotes que chega no router para a saída apropriada; 
Ação local no router
Analogia: 
O routing e planeamento de uma viagem
O forwarding e a passar um cruzamento 
Routing vs forwarding 
algoritmo de routing → Determina a rota através da rede
tabel

In [7]:
llm = OllamaLLM(model="llama3.2")
def query_vector_db(collection,query,top_k=3):
    query_embeddings = model.encode(query).tolist()
    result=collection.query(query_embeddings=[query_embeddings],n_results=top_k)
    relevant_text=result['documents'][0]
    context="\n".join(relevant_text)
    prompt=f"Answer the following question with this material:\n{context}\n Question: {query}"
    return llm.invoke(prompt)

In [8]:
if __name__ == "__main__":
    file_path = "T04_Camada_Rede.pdf"
    text = get_pdf(file_path)
    chunks = chunking(text)
    embeddings = transformer(chunks)
    collection = vector_db(chunks, embeddings)

    intrebare = "Care este subiectul principal al documentului?"
    raspuns = query_vector_db(collection, intrebare)

    with open("respons.txt", "w") as f:
        f.write(raspuns)


created collection


/var/folders/cb/yp5287ws4zbgvbcyhyvlt4kc0000gn/T/ipykernel_9983/2882897658.py:8: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm(prompt)


In [15]:
client=chromadb.Client()

def hash_text(text):
    return hashlib.sha256(text.encode('utf-8')).hexdigest()
def vector_db(chunks, chunk_transformed):
    try:
        collection = client.get_collection('my_collection')
        print("got collection")
    except:
        collection = client.create_collection('my_collection')
        print("created collection")

    existing = set()
    results = collection.get(include=["metadatas"])
    metadatas = results.get("metadatas", [])

    for m in metadatas:
        if m and "hash" in m:
            existing.add(m["hash"])

    ids=[]
    new_chunks=[]
    new_chunk_transformed=[]
    metadatas=[]

    for chunk, emb in zip(chunks, chunk_transformed):
        hash=hash_text(chunk)
        if hash in existing:
            continue
        ids.append(str(uuid.uuid4()))
        new_chunks.append(chunk)
        new_chunk_transformed.append(emb)
        metadatas.append({"hash":hash})


    collection.add(
        documents=new_chunks,
        embeddings=new_chunk_transformed,
        ids=ids,
        metadatas=metadatas
    )

    return collection

In [17]:
text=get_pdf('Economia Para o Sucesso_Manual_2024_2025 (1).pdf')
chunks=chunking(text)
chunks_transformed=transformer(chunks)
vector=vector_db(chunks,chunks_transformed)

print(vector)

got collection
Collection(name=my_collection)
